In [1]:
# 1. scrape today's games with tipoff time (@ like 12pm)
# 2. create a custom yml to run 30 minutes before each of the games on this day
# 3. the script -- for all relevant games -- calls the odds api to fetch and save odds

In [31]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import json
from datetime import datetime, timedelta, timezone
import pytz
from zoneinfo import ZoneInfo
from time import sleep

from scrape import get_first_basket, getId


season = 2025

In [3]:
today = datetime.now(ZoneInfo('America/New_York'))

In [4]:
url = f'https://www.basketball-reference.com/leagues/NBA_{season}_games-{today.strftime("%B").lower()}.html'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'lxml')

In [5]:
table = soup.find('table')
while table.find_all('tr', class_ = 'thead') :
    table.find('tr', class_ = 'thead').decompose()
games = pd.read_html(str(table))[0].rename(columns = {'Start (ET)': 'Time'})
games['Home'] = [x['href'].split('/')[2] for x in table.find_all('a', href = True) if 'teams' in x['href']][1::2]
games['Away'] = [x['href'].split('/')[2] for x in table.find_all('a', href = True) if 'teams' in x['href']][0::2]
games['Date'] = pd.to_datetime(games['Date'])
games = games[pd.to_datetime(games['Date']) == pd.to_datetime(today.date())].reset_index(drop = True)
games['Time'] = (games['Date'].astype(str) + ' ' +  games['Time']).apply(lambda x: datetime.strptime(x.upper() + 'M', "%Y-%m-%d %I:%M%p"))
games['game_id'] = games['Date'].apply(lambda x: datetime.strftime(x, "%Y%m%d")) + '0' + games['Home']

games = games[['game_id', 'Date', 'Time', 'Home', 'Away']]
games

,game_id,Date,Time,Home,Away
0,202411220PHI,2024-11-22,2024-11-22 19:00:00,PHI,BRK
1,202411220WAS,2024-11-22,2024-11-22 19:00:00,WAS,BOS
2,202411220NOP,2024-11-22,2024-11-22 19:30:00,NOP,GSW
3,202411220CHI,2024-11-22,2024-11-22 20:00:00,CHI,ATL
4,202411220HOU,2024-11-22,2024-11-22 20:00:00,HOU,POR
5,202411220MIL,2024-11-22,2024-11-22 20:00:00,MIL,IND
6,202411220DEN,2024-11-22,2024-11-22 22:00:00,DEN,DAL
7,202411220LAC,2024-11-22,2024-11-22 22:30:00,LAC,SAC


In [6]:
games

,game_id,Date,Time,Home,Away
0,202411220PHI,2024-11-22,2024-11-22 19:00:00,PHI,BRK
1,202411220WAS,2024-11-22,2024-11-22 19:00:00,WAS,BOS
2,202411220NOP,2024-11-22,2024-11-22 19:30:00,NOP,GSW
3,202411220CHI,2024-11-22,2024-11-22 20:00:00,CHI,ATL
4,202411220HOU,2024-11-22,2024-11-22 20:00:00,HOU,POR
5,202411220MIL,2024-11-22,2024-11-22 20:00:00,MIL,IND
6,202411220DEN,2024-11-22,2024-11-22 22:00:00,DEN,DAL
7,202411220LAC,2024-11-22,2024-11-22 22:30:00,LAC,SAC


In [ ]:
def datetime_to_cron_utc(dt) :

    time_et = pytz.timezone('US/Eastern').localize(dt) # Localize in ET
    # Convert to UTC
    time_utc = time_et.astimezone(pytz.utc)

    return f"{time_utc.minute} {time_utc.hour} {time_utc.day} {time_utc.month} *"

In [27]:
# Each unique tipoff time minus 30 minutes we want to run the script
execute_crons = [datetime_to_cron_utc(t - timedelta(minutes = 30)) for t in games['Time'].unique()]
execute_crons

['30 23 22 11 *', '0 0 23 11 *', '30 0 23 11 *', '30 2 23 11 *', '0 3 23 11 *']

In [30]:
# .yml write path
yml_path = ".github/workflows/schedule_today_games.yml"  # Output workflow file

# Create the dynamic .yml file content
workflow_content = f"""name: Run Script Before NBA Games

on:
  schedule:
"""

# Add all calculated cron expressions to the workflow
for cron in execute_crons :
    workflow_content += f"    - cron: '{cron}'\n"

workflow_content += """
jobs:
  run_script:
    runs-on: ubuntu-latest
    steps:
      - name: Checkout Repository
        uses: actions/checkout@v3

      - name: Set Up Python
        uses: actions/setup-python@v4
        with:
          python-version: '3.x'

      - name: Install Dependencies
        run: pip install -r requirements.txt

      - name: Run Before Game Script
        run: python run_before_game.py
"""

# Save the workflow content to a .yml file
with open(yml_path, "w") as f:
    f.write(workflow_content)

print(f"Workflow file {yml_path} created successfully!")


Workflow file .github/workflows/schedule_today_games.yml created successfully!


In [38]:
api_key = 'a7bfde5bb651ac64e61f99c67631ef47'

In [ ]:
tmrw_utc = (today + timedelta(days = 1)).astimezone(pytz.utc)

In [39]:
events_response = requests.get(f'https://api.the-odds-api.com/v4/sports/basketball_nba/events',
                               params = {'apiKey': api_key,
                                         'commenceTimeTo': tmrw_utc.strftime('%Y-%m-%dT%H:%M:%SZ')})

In [54]:
events = pd.DataFrame(events_response.json()).rename(columns = {'id': 'event_id'})
events['commence_time'] = pd.to_datetime(events['commence_time'])

In [55]:
with open('utils/odds_tm_map.json', 'r') as file:
    odds_tm_map = json.load(file)

In [59]:
games

,game_id,Date,Time,Home,Away
0,202411220PHI,2024-11-22,2024-11-22 19:00:00,PHI,BRK
1,202411220WAS,2024-11-22,2024-11-22 19:00:00,WAS,BOS
2,202411220NOP,2024-11-22,2024-11-22 19:30:00,NOP,GSW
3,202411220CHI,2024-11-22,2024-11-22 20:00:00,CHI,ATL
4,202411220HOU,2024-11-22,2024-11-22 20:00:00,HOU,POR
5,202411220MIL,2024-11-22,2024-11-22 20:00:00,MIL,IND
6,202411220DEN,2024-11-22,2024-11-22 22:00:00,DEN,DAL
7,202411220LAC,2024-11-22,2024-11-22 22:30:00,LAC,SAC


In [57]:
events['game_id'] = today.strftime("%Y%m%d") + '0' + events['home_team'].map(odds_tm_map)

In [61]:
pd.merge(games, events[['game_id', 'event_id', 'home_team', 'away_team']],
         on = 'game_id')

,game_id,Date,Time,Home,Away,event_id,home_team,away_team
0,202411220PHI,2024-11-22,2024-11-22 19:00:00,PHI,BRK,472f1e3e2efd7d64a3af836936a83ec5,Philadelphia 76ers,Brooklyn Nets
1,202411220WAS,2024-11-22,2024-11-22 19:00:00,WAS,BOS,b600186d2dc2ace2fa5ac10c04e17059,Washington Wizards,Boston Celtics
2,202411220NOP,2024-11-22,2024-11-22 19:30:00,NOP,GSW,9cc15bb04622261a10545f179a61c3a8,New Orleans Pelicans,Golden State Warriors
3,202411220CHI,2024-11-22,2024-11-22 20:00:00,CHI,ATL,feff9ee9f3337c34943c5ae257f40c5e,Chicago Bulls,Atlanta Hawks
4,202411220HOU,2024-11-22,2024-11-22 20:00:00,HOU,POR,d5867a7b248d1fa208569ce6399e662c,Houston Rockets,Portland Trail Blazers
5,202411220MIL,2024-11-22,2024-11-22 20:00:00,MIL,IND,cae716a9887d0ec78c9daee644d94f1c,Milwaukee Bucks,Indiana Pacers
6,202411220DEN,2024-11-22,2024-11-22 22:00:00,DEN,DAL,9e56472e15b270db1994636e56e96a90,Denver Nuggets,Dallas Mavericks
7,202411220LAC,2024-11-22,2024-11-22 22:30:00,LAC,SAC,f1f07b258350717d5f038c3cea64a258,Los Angeles Clippers,Sacramento Kings


In [4]:
url = 'https://www.rotowire.com/basketball/nba-lineups.php'

In [5]:
page = requests.get(url)

In [19]:
url = "https://www.rotowire.com/basketball/nba-lineups.php"
response = requests.get(url)
html = response.text.replace("<!--", "").replace(
    "-->", ""
)  # uncomment tables
soup = BeautifulSoup(html, "lxml")

In [20]:
len([x for x in soup.find_all('table')])

1

In [23]:
pd.read_html(str(table))[0]

,0,1,2
0,📱NBA Sportsbook,🎁 NBA Bonus,🤑 NBA Promo Code
1,📱BetMGM,"🎁 $1,500 First Bet Offer",🤑 ROTOBONUS
2,📱Caesars Sportsbook,"🎁 $1,000 First Bet on Caesars",🤑 ROTO1000
3,📱Fanatics Sportsbook,"🎁 Bet & Get $1,000 in No Sweat Bets",🤑 CLICK HERE
4,📱DraftKings,"🎁 Bet $5, Win $150 In Bonus Bets",🤑 CLICK HERE
5,📱FanDuel,"🎁 Bet $5, Win $150 in Bonus Bets + 3 Months of...",🤑 CLICK HERE
6,📱bet365,"🎁 $1,000 First Bet Safety Net or Bet $5, Get $150",🤑 ROTOWIRE


In [24]:
soup.find_all('button', class_ = 'see-proj-minutes')

[<button class="see-proj-minutes" data-home="0" data-nickname="Bulls" data-team="CHI">Projected Minutes</button>,
 <button class="see-proj-minutes" data-home="1" data-nickname="Pistons" data-team="DET">Projected Minutes</button>,
 <button class="see-proj-minutes" data-home="0" data-nickname="76ers" data-team="PHI">Projected Minutes</button>,
 <button class="see-proj-minutes" data-home="1" data-nickname="Heat" data-team="MIA">Projected Minutes</button>,
 <button class="see-proj-minutes" data-home="0" data-nickname="Wizards" data-team="WAS">Projected Minutes</button>,
 <button class="see-proj-minutes" data-home="1" data-nickname="Knicks" data-team="NYK">Projected Minutes</button>,
 <button class="see-proj-minutes" data-home="0" data-nickname="Pacers" data-team="IND">Projected Minutes</button>,
 <button class="see-proj-minutes" data-home="1" data-nickname="Raptors" data-team="TOR">Projected Minutes</button>,
 <button class="see-proj-minutes" data-home="0" data-nickname="Rockets" data-team

In [44]:
lineup_away = soup.find('ul', class_ = 'lineup__list is-visit')
starting_lineup_tags = lineup_away.find_all('li')[1:6]
[x.text.split('\n')[-2] for x in starting_lineup_tags]

['Josh Giddey', 'Coby White', 'Zach LaVine', 'P. Williams', 'N. Vucevic']

In [47]:
lineup_home = soup.find('ul', class_ = 'lineup__list is-home')
starting_lineup_tags = lineup_home.find_all('li')[1:6]
[x.text.split('\n')[-2] for x in starting_lineup_tags]

['C. Cunningham', 'Jaden Ivey', 'T. Hardaway', 'Tobias Harris', 'Jalen Duren']

In [48]:
[x for x in starting_lineup_tags.find_all('a', href = True)]

AttributeError: 'list' object has no attribute 'find_all'

In [50]:
[x['href'] for x in lineup_home.find_all('a', href = True)]

['/basketball/player/cade-cunningham-5336',
 '/basketball/player/jaden-ivey-5696',
 '/basketball/player/tim-hardaway-3459',
 '/basketball/player/tobias-harris-3211',
 '/basketball/player/jalen-duren-5703',
 '/basketball/player/tim-hardaway-3459',
 '/basketball/player/jaden-ivey-5696',
 '/basketball/player/bobi-klintman-5976',
 '/basketball/player/ausar-thompson-5932']

In [1]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from pretty_html_table import build_table
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame(np.array([[0,1], [2, 3]]), columns = ['col1', 'col2'])

In [3]:
# Specify the email contents
mail = MIMEMultipart()
html = """\
<html><head></head><body>{0}</body></html>
""".format(build_table(df, 'grey_light', text_align = 'right', font_family = 'arial', width_dict = ['100','200','200','100','100','100','100'], font_size = 10))
mail.attach(MIMEText(html, 'html'))


In [4]:

# Set my email address and the password key
my_mail  = 'martinbog19@gmail.com'
# with open('gmail_key.txt') as f:
#     password = f.read()

# Set the subject of the email
mail['Subject'] = 'test_df'


In [7]:

# Connect to the Gmail SMTP server & log in
server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login('martinbogbog@gmail.com', "Kunaguero16")

SMTPAuthenticationError: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials 6a1803df08f44-6d40dbe2324sm44517326d6.12 - gsmtp')

In [ ]:

# Send the email (to myself)
server.sendmail(my_mail, my_mail, mail.as_string())
q = server.quit()

Email sent successfully!


In [9]:
import os

def is_running_in_github_actions():
    return os.getenv("GITHUB_ACTIONS") == "true"

In [12]:
os.getenv("k")